In [20]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Paths
train_dir = '/content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/train'  # Path to train folder
test_dir = '/content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/test'    # Path to test folder



In [21]:
# Debug: Check directory structure
print("Train Directory Structure:")
for root, dirs, files in os.walk(train_dir):
    print(f"Root: {root}, Subfolders: {dirs}, Files: {len(files)}")
    if len(files) == 0:
        print(f"Warning: No files in folder {root}")

print("Test Directory Content:")
for root, dirs, files in os.walk(test_dir):
    print(f"Root: {root}, Subfolders: {dirs}, Files: {len(files)}")
    if len(files) == 0:
        print(f"Warning: No files in folder {root}")

Train Directory Structure:
Root: /content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/train, Subfolders: ['Boots', 'Slippers', 'Sandals'], Files: 0
Root: /content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/train/Boots, Subfolders: [], Files: 326
Root: /content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/train/Slippers, Subfolders: [], Files: 146
Root: /content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/train/Sandals, Subfolders: [], Files: 306
Test Directory Content:
Root: /content/drive/MyDrive/Colab Notebooks/CNN/Shoe Classification/test, Subfolders: [], Files: 89


In [22]:
# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 3

# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

In [23]:
# Training and Validation Generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 623 images belonging to 3 classes.


In [24]:
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)


Found 155 images belonging to 3 classes.


In [25]:
# Test Generator (no labels, shuffle=False to maintain file order)
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode=None,
    shuffle=False,
    classes=['class1', 'class2', 'class3']
)

Found 0 images belonging to 3 classes.


In [26]:
# Pretrained Model (Transfer Learning)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model layers

# Custom Model Head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')  # Output for 3 categories
])

In [27]:
# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the Model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=train_gen.samples // BATCH_SIZE,
    validation_steps=val_gen.samples // BATCH_SIZE
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 162s 8s/step - accuracy: 0.4279 - loss: 1.3258 - val_accuracy: 0.4844 - val_loss: 0.9553
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 474ms/step - accuracy: 0.5312 - loss: 1.0713 - val_accuracy: 0.5556 - val_loss: 0.8521
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 148s 8s/step - accuracy: 0.5560 - loss: 0.8933 - val_accuracy: 0.6172 - val_loss: 0.8417
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 347ms/step - accuracy: 0.6562 - loss: 0.8374 - val_accuracy: 0.6667 - val_loss: 0.7750
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 147s 8s/step - accuracy: 0.6667 - loss: 0.7809 - val_accuracy: 0.6328 - val_loss: 0.7568
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 324ms/step - accuracy: 0.7188 - loss: 0.6849 - val_accuracy: 0.7037 - val_loss: 0.7020
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 189s 8s/step - accuracy: 0.6621 - loss: 0.7362 - val_accuracy: 0.6328 - val_loss: 0.7685
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 16s 492ms/step - accuracy: 0.5938 - loss: 0.7569 - val_accuracy: 0.7037 - val_

In [28]:
# Evaluate on Validation Data
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")


5/5 ━━━━━━━━━━━━━━━━━━━━ 32s 6s/step - accuracy: 0.6131 - loss: 0.7469
Validation Accuracy: 63.87%


In [30]:
# Predict Labels for Test Data
test_gen.reset()  # Ensure generator starts from the beginning
predictions = model.predict(val_gen, verbose=1)
predicted_labels = np.argmax(predictions, axis=1)


5/5 ━━━━━━━━━━━━━━━━━━━━ 35s 6s/step


In [31]:
# Map Predictions to Class Labels
class_indices = train_gen.class_indices
class_labels = {v: k for k, v in class_indices.items()}  # Reverse the dictionary
predicted_classes = [class_labels[label] for label in predicted_labels]

In [32]:
# Display Predictions
for i, filename in enumerate(test_gen.filenames):
    print(f"Filename: {filename}, Predicted Label: {predicted_classes[i]}")